<a href="https://colab.research.google.com/github/Gagalin105/PL-Repo.-README/blob/main/HW1%EF%BC%9A%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E8%88%87%E5%B9%B3%E8%A1%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

日常支出速算與分攤（作業一）


*   輸入種類與數值
*   根據種類分類


*   計算總收入





# **標頭檔**


In [ ]:
import pandas as pd
from datetime import datetime
import re
import gspread
from google.colab import auth
from google.auth import default

# Authenticate and authorize gspread
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Initialize global variables
total_income = 0

# **函式：輸入**

In [19]:
def get_expense_input():
    """Gets expense details from the user."""
    date_input = input("請輸入日期 (YYYY-MM-DD): ")
    time_input = input("請輸入時間 (HH:MM): ")
    item_input = input("請輸入項目: ")
    amount_input_str = input("請輸入金額: ")
    category_input = input("請根據先前輸入的項目分類，並輸入對應的數字：1. 收入、2. 吃喝、3. 娛樂: ")

    try:
        amount_input = float(amount_input_str)
    except ValueError:
        print("無效的金額輸入。請輸入數字。")
        return None, None, None, None, None # Return None for all if amount is invalid

    return date_input, time_input, item_input, amount_input, category_input

# Modify the add_expense function to accept parameters
def add_expense(date_str, time_str, item, amount, category_input):
    try:
        # Validate and correct date format (YYYY-MM-DD)
        date_match = re.match(r'(\d{4})[=/-](\d{1,2})[=/-](\d{1,2})', date_str)
        if not date_match:
            print("日期格式錯誤：請使用 YYYY-MM-DD 格式。")
            return

        year, month, day = date_match.groups()
        # Attempt to correct month and day if they are single digits
        corrected_date_str = f"{year}-{int(month):02d}-{int(day):02d}"

        # Validate and correct time format (HH:MM)
        time_match = re.match(r'^(\d{1,2}):(\d{1,2})$', time_str)
        if not time_match:
            print("時間格式錯誤：請使用 HH:MM 格式 (24小時制)。")
            return

        hour, minute = time_match.groups()
        # Attempt to correct hour and minute if they are single digits and validate values
        try:
            corrected_time_str = f"{int(hour):02d}:{int(minute):02d}"
            # Further validation for hour and minute values
            if not (0 <= int(hour) <= 23 and 0 <= int(minute) <= 59):
                 print("時間格式錯誤：請使用 HH:MM 格式 (24小時制)。時或分超出有效範圍。")
                 return
        except ValueError:
             print("時間格式錯誤：請使用 HH:MM 格式 (24小時制)。")
             return

        # Combine corrected date and time strings and convert to datetime object
        datetime_str = f"{corrected_date_str} {corrected_time_str}"
        datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M')

        # --- Category input and validation (moved from original add_expense) ---
        category_map = {'1': '收入', '2': '吃喝', '3': '娛樂'}

        if category_input not in category_map:
            print("無效的分類輸入。請輸入 1, 2, 或 3。")
            return

        category = category_map[category_input]
        # --- End of category validation ---


        # Create a new row as a list for gspread
        # Add the category to the list
        new_row_list = [datetime_obj.date().strftime('%Y-%m-%d'),
                        datetime_obj.time().strftime('%H:%M'),
                        item,
                        amount,
                        category] # Add the category here


        # Append the new row to the Google Sheet using gspread
        # Use the gc object from the previous cell
        sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hWNTg2T0acFyg1xrtLd35HZJxOQFrg0bVyBdgcWYTJ4/edit?gid=0#gid=0")
        worksheet = sheet.sheet1 # Assuming the data is in the first sheet

        worksheet.append_row(new_row_list)

        print("Row added to Google Sheet successfully!")

        # --- New code for updating total income ---
        global total_income
        if category == '收入':
            total_income += amount
        else:
            total_income -= amount

        print(f"目前的總收入為 {total_income} 元")
        # --- End of new code ---


        # Optional: Update the DataFrame in memory after writing to the sheet
        global df
        if 'df' in globals() and isinstance(df, pd.DataFrame):
            new_row_dict = {'日期': datetime_obj.date(),
                            '時間': datetime_obj.time(),
                            '項目': item,
                            '金額': amount,
                            '類型': category} # Add category to DataFrame dict
            # Use pd.concat instead of append for newer pandas version

    except ValueError as e:
        print(f"Error adding row: {e}")
        print("請確保日期和時間格式正確。")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Example of how to use the functions
# date_input, time_input, item_input, amount_input, category_input = get_expense_input()
# if all([date_input, time_input, item_input, amount_input is not None, category_input]): # Check if all inputs are valid
#     add_expense(date_input, time_input, item_input, amount_input, category_input)

# **顯示時依時間排序**


In [20]:
# Combine '日期' and '時間' into a single datetime column
df['時間戳記'] = df['日期'] + ' ' + df['時間']

# Convert the '時間戳記' column to datetime objects
df['時間戳記'] = pd.to_datetime(df['時間戳記'])

# Sort the DataFrame by the new '時間戳記' column in ascending order
df = df.sort_values(by='時間戳記', ascending=True)

# Display the first few rows of the sorted DataFrame to verify
display(df.head())

NameError: name 'df' is not defined

## 顯示排序後的項目

### Subtask:
將排序後的 DataFrame 顯示給使用者。


**Reasoning**:
Add a text description before displaying the sorted DataFrame and then display the entire DataFrame to the user.



In [ ]:
print("以下是依時間排序的所有記帳項目：")
display(df)

# **函式：menu**

In [ ]:
def display_menu():
    """Displays the main menu options to the user."""
    print("\n請選擇功能：")
    print("1. 新增支出/收入")
    print("2. 顯示所有記帳項目 (依時間排序)")
    print("3. 顯示財務總結")
    print("q. 退出")

# **函式：顯示儲存資料**

In [21]:
def display_all_items():
    """Reads, sorts, and displays all items from the Google Sheet."""
    # Access the Google Sheet using the gc object and the sheet URL
    sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hWNTg2T0acFyg1xrtLd35HZJxOQFrg0bVyBdgcWYTJ4/edit?gid=0#gid=0")
    worksheet = sheet.sheet1

    # Read all data from the worksheet
    data = worksheet.get_all_values()

    if len(data) > 1:
        df = pd.DataFrame(data[1:], columns=data[0])

        # Ensure '金額' column is numeric, coercing errors to NaN
        df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
        df.dropna(subset=['金額'], inplace=True)

        # Ensure '日期' and '時間' columns exist
        if '日期' in df.columns and '時間' in df.columns:
             # Combine '日期' and '時間' into a single datetime column
             df['時間戳記'] = df['日期'] + ' ' + df['時間']
             # Convert the '時間戳記' column to datetime objects, handling potential errors
             df['時間戳記'] = pd.to_datetime(df['時間戳記'], errors='coerce')
             df.dropna(subset=['時間戳記'], inplace=True) # Drop rows with invalid datetime

             # Sort the DataFrame by the new '時間戳記' column in ascending order
             df = df.sort_values(by='時間戳記', ascending=True)

             print("\n以下是依時間排序的所有記帳項目：")
             display(df[['日期', '時間', '項目', '金額', '類型']]) # Display relevant columns
        else:
             print("Google Sheet 中缺少 '日期' 或 '時間' 欄位，無法依時間排序顯示。")
             display(df) # Display unsorted if sorting not possible


    else:
        print("Google Sheet 是空的，沒有記帳項目。")

# **函式：財務總結**

In [ ]:
def display_financial_summary():
    """Reads data and displays the financial summary."""
    try:
        # Access the Google Sheet and read data (similar to display_all_items)
        sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hWNTg2T0acFyg1xrtLd35HZJxOQFrg0bVyBdgcWYTJ4/edit?gid=0#gid=0")
        worksheet = sheet.sheet1
        data = worksheet.get_all_values()

        if len(data) > 1:
            df = pd.DataFrame(data[1:], columns=data[0])

            # Ensure '金額' column is numeric and '類型' column exists
            df['金額'] = pd.to_numeric(df['金額'], errors='coerce')
            df.dropna(subset=['金額'], inplace=True)
            if '類型' not in df.columns:
                df['類型'] = '未知' # Default category if column is missing

            # Calculate total income and expenses
            total_income = df[df['類型'] == '收入']['金額'].sum()
            total_expense = df[df['類型'] != '收入']['金額'].sum()
            total_balance = total_income - total_expense

            print("\n財務總結:")
            print(f"總收入: {total_income:.2f} 元")
            print(f"總支出: {total_expense:.2f} 元")
            print(f"總餘額: {total_balance:.2f} 元")
        else:
            print("Google Sheet 是空的，沒有記帳項目可供計算。")
            print("\n財務總結:")
            print("總收入: 0.00 元")
            print("總支出: 0.00 元")
            print("總餘額: 0.00 元")

    except Exception as e:
        print(f"計算或顯示財務總結時發生錯誤: {e}")

    return True


# **主迴圈**

In [22]:
while True:
    display_menu()
    choice = input("請輸入您的選擇: ").lower()

    if choice == '1':
        date_input, time_input, item_input, amount_input, category_input = get_expense_input()
        if all([date_input, time_input, item_input, amount_input is not None, category_input]):
            add_expense(date_input, time_input, item_input, amount_input, category_input)
        else:
            print("輸入無效，請重試。")
    elif choice == '2':
        display_all_items()
    elif choice == '3':
        display_financial_summary()
    elif choice == 'q':
        print("程式結束。")
        break
    else:
        print("無效的選擇，請重新輸入。")


請選擇功能：
1. 新增支出/收入
2. 顯示所有記帳項目 (依時間排序)
3. 顯示財務總結
q. 退出
請輸入您的選擇: 2


NameError: name 'gc' is not defined